# Google Earth Engine Panel Data Creation

## Initialize

In [1]:
# !pip install geemap
#!pip install ee

In [2]:
# !pip install uszipcode

In [3]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time


In [4]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [5]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()

## Load Feature Collection - Shapefiles

In [6]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sc_county = counties.filter(ee.Filter.eq('NAME', 'Santa Clara'))

In [7]:
la_county, sc_county

(<ee.featurecollection.FeatureCollection at 0x7f7ee69cf640>,
 <ee.featurecollection.FeatureCollection at 0x7f7ee69cf250>)

In [8]:
#Income Data
la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")

## Load NAIP Imagery

In [9]:
def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image
     

In [10]:
def get_images(param_dict):
    source_image_collection = params['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

    
    

## Load Labeled Data

In [16]:
## Loading feature collections from Google Earth Engine

#water = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance")
water_training = ee.FeatureCollection("projects/california-lawn-detection/assets/water_training")
trees_training = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_training")
grass_training = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_training")
turf_training = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_training")
#pv_training = ee.FeatureCollection("projects/california-lawn-detection/assets/pv_training")
impervious_training = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_training").limit(50)
soil_training = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_training").limit(50)

LABELED_SET = water_training.merge(trees_training).merge(grass_training).merge(turf_training).merge(impervious_training).merge(soil_training)



In [17]:
water_test = ee.FeatureCollection("projects/california-lawn-detection/assets/water_test")
vegetation_trees_test = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_test")
vegetation_grass_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_test")
turf_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_test")
#pv_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/pv_test")
impervious_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_test")
soil_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_test")

TEST_SET = water_test.merge(vegetation_trees_test).merge(vegetation_grass_test).merge(turf_test).merge(impervious_test).merge(soil_test)



## Build Training Set

In [18]:
training_image_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020],
        'counties': {'la_county': la_county}
         }

TRAINING_IMAGE = get_images(training_image_params)['2020_la_county']

In [19]:
Map.addLayer(TRAINING_IMAGE, {}, 'TRAINING_IMAGE')

In [20]:
# Overlay the points on the imagery to get training.
LABEL = 'landcover'
BANDS = ['R', 
         'G', 
         'B', 
         'N', 
         'NDVI',
         'R_Entropy',
         'R_Contrast',
         'R_Gearys',
         'G_Entropy',
         'G_Contrast',
         'G_Gearys',
         'B_Entropy',
         'B_Contrast',
         'B_Gearys',
         'N_Entropy', 
         'N_Contrast', 
         'N_Gearys']

train_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': LABELED_SET,
  'properties': [LABEL],
  'scale': 1
})

test_data = TRAINING_IMAGE.select(BANDS).sampleRegions(**{
  'collection': TEST_SET,
  'properties': [LABEL],
  'scale': 1
})

In [21]:
set(BANDS)==set(TRAINING_IMAGE.bandNames().getInfo())

True

In [22]:
# print("Training Set Size in Pixels", train_data.aggregate_count('R').getInfo())

In [23]:
# print("Test Set Size in Pixels", test_data.aggregate_count('R').getInfo())

## Machine Learning Model

In [24]:
clf = ee.Classifier.smileRandomForest(numberOfTrees = 200, minLeafPopulation = 5, bagFraction= 0.7)\
                   .train(train_data, LABEL, BANDS)
clf

In [25]:
training_image_classified = TRAINING_IMAGE.select(BANDS)\
                                          .classify(clf)


In [26]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

Map.addLayer(training_image_classified, {'min': 1, 'max': 7, 'palette': legend_colors}, 'Classification')

In [27]:
training_image_classified.bandNames().getInfo()

['classification']

In [28]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

## Binary Classification and Area Calculation

In [33]:
def area_calculation(image, class_number, shape, pixel_scale = 20):

    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))

    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())

    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = shape,
        scale = pixel_scale,
        maxPixels = 1e13)


    area_sq_m = area.getInfo().get('classification')

    area_sq_km = area_sq_m / 1e6

    return area_sq_km

In [34]:
def ndvi_calculation(image, class_number, shape, ref_image, pixel_scale=1):
    
    if type(shape) == str:
        shape = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', shape))
        
    ndvi = ref_image.normalizedDifference(['N', 'R'])
    image_clipped = image.clip(shape)
    
    NDVI_for_class = ndvi.updateMask(image_clipped.select('classification').eq(class_number))
    
    reducer = ee.Reducer.mean()\
                        .combine(ee.Reducer.max(),sharedInputs=True)\
                        .combine(ee.Reducer.min(),sharedInputs=True)
    
    
    qty = NDVI_for_class.reduceRegion(
        reducer = reducer, 
        geometry = shape, 
        scale = pixel_scale, 
        maxPixels = 1e13)
    return qty



### Create Panel Data

In [35]:
#import parcel shapes so we can clip by residential areas

la_parcel_shape_filtered = ee.FeatureCollection("projects/california-lawn-detection/assets/LA_County_Parcels_Shape")\
                             .filter(ee.Filter.eq('UseType', 'Residential'))
    
la_parcel_res = la_parcel_shape_filtered.select(ee.List(['AIN', 'SitusCity','SitusZIP','SitusFullA']), 
                                                ee.List(['AIN', 'City','ZipCode','FullAddress']))


In [36]:
#import zipcode shapes so we can clip by zipcodes

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [50]:
def run_inference(inference_params):
    
    inference_images = get_images(inference_params)
    residential = inference_params['residential']
    zipcode_list = inference_params['zipcodes']
    ndvi = inference_params['ndvi']
    zipcode_shape = inference_params['zipcode_shape']
    residential_shape = inference_params['residential_shape']
    
    base_keys = ['year','polygon','water_area','vegetation_trees_area', 
        'vegetation_grass_area', 'turf_area', 'impervious_area',
        'soil_area', 'total_area']
    
    ndvi_keys = ['tree_ndvi_mean', 'tree_ndvi_max','tree_ndvi_min',
       'grass_ndvi_mean', 'grass_ndvi_max','grass_ndvi_min']
    
    dictionary = {}
    
    if inference_params['residential']:
        print('CLIPPING AREA TO INCLUDE RESIDENTIAL AREAS ONLY')
    if inference_params['ndvi']:
        print('RUNNING INFERENCE INCLUDING NDVI CALCULATIONS')
    if inference_params['residential'] or inference_params['ndvi']:
        print('---------------------------------------------------------------------')
    
    for i in base_keys:
        dictionary[i] = []
    if ndvi:
        for i in ndvi_keys:
            dictionary[i] = []
        
    for i in zipcode_list:
        for j in list(inference_images.items()):
            im = j[1]
            if residential:
                im = im.clip(residential_shape)
            imagery = im.select(BANDS).classify(clf)
            name = j[0]

            start = time()
            polygon = zipcode_shape.filter(ee.Filter.eq('ZipCode', i))

            dictionary['year'].append(j[0][:4]) 
            dictionary['polygon'].append(i)

            water_area = area_calculation(imagery, 0, polygon, 20)
            dictionary['water_area'].append(water_area)

            vegetation_trees_area = area_calculation(imagery, 1, polygon, 20)
            dictionary['vegetation_trees_area'].append(vegetation_trees_area)

            vegetation_grass_area = area_calculation(imagery, 2, polygon, 20)
            dictionary['vegetation_grass_area'].append(vegetation_grass_area)

            turf_area = area_calculation(imagery, 3, polygon, 20)
            dictionary['turf_area'].append(turf_area)

            impervious_area = area_calculation(imagery, 4, polygon, 20)
            dictionary['impervious_area'].append(impervious_area)

            soil_area = area_calculation(imagery, 5, polygon, 20)
            dictionary['soil_area'].append(soil_area)

            total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area
            dictionary['total_area'].append(total_area)
            
            if ndvi:
                tree_ndvi_mean, tree_ndvi_max, tree_ndvi_min = ndvi_calculation(imagery, 1, polygon, ref_image = im).getInfo().values()
                dictionary['tree_ndvi_mean'].append(tree_ndvi_mean)
                dictionary['tree_ndvi_max'].append(tree_ndvi_max)
                dictionary['tree_ndvi_min'].append(tree_ndvi_min)

                grass_ndvi_mean, grass_ndvi_max, grass_ndvi_min = ndvi_calculation(imagery, 2, polygon, ref_image = im).getInfo().values()
                dictionary['grass_ndvi_mean'].append(grass_ndvi_mean)
                dictionary['grass_ndvi_max'].append(grass_ndvi_max)
                dictionary['grass_ndvi_min'].append(grass_ndvi_min)



            end = time()
            print(f'Zip Code: {i}, Year: {j[0][:4]} ::: completed in {end-start} seconds.')
    return dictionary
              
              

In [51]:
inference_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2018,2020],
        'zipcodes': ['90802','90732'],
        'ndvi': False,
        'residential': False,
        'residential_shape': la_parcel_res, #don't adjust this line
        'counties': {'la_county': la_county}, #don't adjust this line
        'zipcode_shape' : la_county_income_zipcode #don't adjust
         }

In [52]:
dictionary = run_inference(inference_params)

Zip Code: 90802, Year: 2018 ::: completed in 28.381202936172485 seconds.
Zip Code: 90802, Year: 2020 ::: completed in 49.04420208930969 seconds.
Zip Code: 90732, Year: 2018 ::: completed in 36.01553392410278 seconds.
Zip Code: 90732, Year: 2020 ::: completed in 38.734976053237915 seconds.


In [53]:
df = pd.DataFrame(dictionary)
df

,year,polygon,water_area,vegetation_trees_area,vegetation_grass_area,turf_area,impervious_area,soil_area,total_area
0,2018,90802,0.400216,0.147730,0.153992,0.226933,13.764950,0.062795,14.756617
1,2020,90802,0.000000,0.303413,0.291956,1.127961,9.826548,3.206738,14.756617
2,2018,90732,0.029005,0.685563,0.771261,0.434912,4.708557,1.602252,8.231549
3,2020,90732,0.000000,1.393330,0.464795,0.486332,1.390442,4.496650,8.231549
